<h1 align="center" style="color:#00FFFF" >NLP Assignment#4</h1>

***
<h1 align="center" style="color:#93C572">Sequence Models weather forecasting</h1>

### Names and Ids:-
*   Abdelrahman Mahmoud Alsayed Ibrahim - 20190732
*   Yousef Yasser Ezzat - 20190652
*   Omar  Mohammed Abdelbary - 20190358


In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("weather_prediction_dataset.csv")
data.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max
0,20000101,1,8,0.89,1.0286,0.20,0.03,0.0,2.9,1.6,...,-9.3,0.7,1.6,0.97,1.0275,0.25,0.04,8.5,7.2,9.8
1,20000102,1,8,0.87,1.0318,0.25,0.00,0.0,3.6,2.7,...,0.5,2.0,2.0,0.99,1.0293,0.17,0.16,7.9,6.6,9.2
2,20000103,1,5,0.81,1.0314,0.50,0.00,3.7,2.2,0.1,...,-1.0,2.8,3.4,0.91,1.0267,0.27,0.00,8.1,6.6,9.6
3,20000104,1,7,0.79,1.0262,0.63,0.35,6.9,3.9,0.5,...,2.5,4.6,4.9,0.95,1.0222,0.11,0.44,8.6,6.4,10.8
4,20000105,1,5,0.90,1.0246,0.51,0.07,3.7,6.0,3.8,...,-1.8,2.9,3.6,0.95,1.0209,0.39,0.04,8.0,6.4,9.5


In [3]:
num_features = [
    "TOURS_wind_speed",
    "TOURS_humidity",
    "TOURS_pressure",
    "TOURS_global_radiation",
    "TOURS_precipitation",
    "TOURS_temp_mean",
    "TOURS_temp_max",
    "DATE",
]
df_full = data[num_features]
df_full.head()

,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_max,DATE
0,1.6,0.97,1.0275,0.25,0.04,8.5,9.8,20000101
1,2.0,0.99,1.0293,0.17,0.16,7.9,9.2,20000102
2,3.4,0.91,1.0267,0.27,0.00,8.1,9.6,20000103
3,4.9,0.95,1.0222,0.11,0.44,8.6,10.8,20000104
4,3.6,0.95,1.0209,0.39,0.04,8.0,9.5,20000105


### Preparing the data and divide it into train and validation
### Dates from 2000 to 2007 used for training your model Dates from 2008 and 2009 used to validate your model.

In [4]:
train = df_full["DATE"] < 20080101

validation = df_full["DATE"] >= 20080101

train = df_full[train]
val = df_full[validation]

y_train = train["TOURS_temp_max"] # target
y_valid = val["TOURS_temp_max"] # target

X_train = train.drop(columns=["DATE"])
X_valid = val.drop(columns=["DATE"])


print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(2922, 7) (732, 7) (2922,) (732,)


### Data Normalization

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_valid = sc.fit_transform(X_valid)

y_train = y_train.values.reshape(-1, 1)
y_valid = y_valid.values.reshape(-1, 1)

y_train = sc.fit_transform(y_train)
y_valid = sc.fit_transform(y_valid)

In [6]:
X_train.shape, y_train.shape

((2922, 7), (2922, 1))

<h1 align="center" style="color:#FF007F">Creating training pipeline</h1>

In [7]:
from keras.models import Sequential
from keras.layers import SimpleRNN,GRU,LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

def rnn(input_shape):
    # Define the RNN model
    model = Sequential(name="RNN")
    model.add(SimpleRNN(64, input_shape=input_shape)) # hidden layer size = 64
    model.add(Dense(1, activation='tanh'))
    return model

def gru(input_shape):
    # Define the GRU model
    model = Sequential(name="GRU")
    model.add(GRU(64, input_shape=input_shape)) # hidden layer size = 64
    model.add(Dense(1, activation='tanh'))
    return model

def lstm(input_shape):
    # Define the LSTM model
    model = Sequential(name="LSTM")
    model.add(LSTM(64, input_shape=input_shape))  # hidden layer size = 64
    model.add(Dense(1, activation="tanh"))
    return model


def reshape_with_time_steps(features, target, timesteps):
    new_features = []
    for i in range(0, len(features) - timesteps, timesteps):
        new_features.append(features[i : i + timesteps])
    new_targets = []
    for i in range(timesteps, len(target), timesteps):
        new_targets.append(target[i])
    return np.array(new_features), np.array(new_targets)


def training_pipeline(
    models: list,
    training_features,
    training_targets,
    validation_features,
    validation_targets,
):
    # Report:
    REPORT = {}
    
    # Hyper-Parameters:-
    step_sizes = [7, 30, 356] # steps for 1 week, 1 month, 1 year
    OPTIMIZER = Adam(learning_rate=0.001)
    early_stop = EarlyStopping(monitor="val_loss", mode='min',patience=10)
    
    # Running the pipeline...
    for model in models:
        for step_size in step_sizes:
            X_train, y_train, X_valid, y_valid = (
                training_features,
                training_targets,
                validation_features,
                validation_targets,
            )
            # Reshaping data with timestep
            X_train, y_train = reshape_with_time_steps(X_train, y_train, step_size)
            print(X_train.shape, y_train.shape)
            X_valid, y_valid = reshape_with_time_steps(X_valid, y_valid, step_size)
            print(X_valid.shape, y_valid.shape)

            # Initializing input shape for model
            INPUT_SHAPE = (step_size, 7)

            # Initiating model
            MODEL = model(INPUT_SHAPE)

            # Compiling and Summary
            MODEL.compile(optimizer=OPTIMIZER, loss="mean_squared_error")
            MODEL.summary()

            # Starting training and showing result
            print(
                "________Started training for Model--> {0} with step size: {1}_________".format(
                    MODEL.name, step_size
                )
            )
            MODEL.fit(
                X_train,
                y_train,
                epochs=50,
                validation_data=(X_valid, y_valid),
                callbacks=[early_stop],verbose=1
            )

            # Evaluate the model on the test set
            test_score = MODEL.evaluate(X_valid, y_valid)
            print("Test loss:", test_score)
            print("________Training Finished________")
            
            # Add Model and training result to the report
            REPORT[str(MODEL.name)+f"  with step_size == {step_size}"] = [test_score,early_stop.stopped_epoch]
            
            
    print(REPORT)            
                      

MODELS = [lstm,gru,rnn]
training_pipeline(MODELS, X_train, y_train, X_valid, y_valid)

(417, 7, 7) (417, 1)
(104, 7, 7) (104, 1)
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-17 01:04:22.027178: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-17 01:04:22.027441: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                18432     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 18,497
Trainable params: 18,497
Non-trainable params: 0
_________________________________________________________________
________Started training for Model--> LSTM with step size: 7_________
Epoch 1/50


2023-05-17 01:04:22.332988: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-17 01:04:22.979777: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-17 01:04:23.124064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/14 [=>............................] - ETA: 14s - loss: 0.8203

...

2023-05-17 01:06:10.132947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 1.4736

2023-05-17 01:06:39.183808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 34s 34s/step - loss: 1.4736 - val_loss: 3.7396
Epoch 2/50
1/1 [==============================] - 33s 33s/step - loss: 0.9852 - val_loss: 3.4838
Epoch 3/50


KeyboardInterrupt: 

<h1 align="left" style="color:#FF33E6"> FINAL REPORT</h1>

#### Note:- 0 in # of epochs means that early stop didn't happen

| STEP SIZE | # of Epochs(early stopping effect) | RNN Validation loss | LSTM Validation loss | GRU Validation loss |
|:---------:|:----------------------------------:|:-------------------:|:--------------------:|:-------------------:|
|     7     |             12,  0,  14             |       0.3076        |        0.2708        |       0.2674        |
|    30     |             21, 25, 25              |       0.1665        |        0.1582        |       0.1932        |
|    356    |             27, 24, 21             |       0.3153        |        0.2978        |       0.2923        |


<h1 align="center" style="color: #DAEF0C">CONCLUSION</h1>

***
Based on the provided statistics, we can draw some conclusions regarding the performance of the RNN, LSTM, and GRU models with different step sizes and early stopping:

### 1.Model Performance:
Across all three step sizes, the LSTM model performed the best with the lowest validation loss, followed by the RNN model and the GRU model. Therefore, the LSTM model is the most suitable model for this particular task.

### 2.Optimal Step Size:
The optimal step size for each model varied. For the LSTM, RNN and GRU models, a step size of 30 resulted in the lowest validation loss.

### 3.Optimal Number of Epochs:
The optimal number of epochs for each model varied depending on the step size. For step size 7, the GRU model achieved the lowest validation loss with 14 epochs, while the RNN and LSTM models achieved the lowest validation loss with 12 and 0 epochs, respectively. For step size 30, the LSTM model achieved the lowest validation loss with 25 epochs, while the RNN and GRU models achieved the lowest validation loss with 21 epochs. For step size 356, the LSTM model achieved the lowest validation loss with 24 epochs, while the GRU and RNN models achieved the lowest validation loss with 21 and 27 epochs, respectively.

### 4.Early Stopping:
The results showed that early stopping had a significant impact on the performance of the models. Specifically, the optimal number of epochs for each model was lower when early stopping was used. This suggests that early stopping should be used to prevent overfitting and improve the performance of the models.



### Summary:
In conclusion, based on the provided statistics, the LSTM model with a step size of 30 and an optimal number of epochs around 25 is the best performing model for this particular task. However, the RNN and GRU models performed well with a step size of 30 and an optimal number of epochs around 21 and 25, respectively. Therefore, the choice of model and step size depends on the specific needs of the task. Additionally, early stopping should be used to optimize the performance of the models and prevent overfitting.